In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb

seed = 42

In [2]:
import nbimporter

import pre_processing
import feature_selection

import feature_generation_dangerous
import feature_generation_reliable

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation_reliable.ipynb
Importing Jupyter notebook from feature_generation_dangerous.ipynb
Importing Jupyter notebook from feature_selection.ipynb


In [3]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

# IDEA: Convertir el problema en un problema de clasificacion
Tras haber trabajado muchos dias con el dataframe original, mandando directamente a entrenar a un modelo de regresion para intentar predecir los precios, hemos llegado a obtener un MAE de 472k, que nos parece muy alto. Una idea que entonces surge, es ver donde estamos fallando por mucho.

Tras realizar el **analisis de error correspondiente**, hemos podido observar que nuestro modelo de regresion esta fallando por mucho en los outliers, propiedades que generan un error absoluto de 10 millones o mas. Esto se debe a datos que no tienen sentido. Para encarar este problema, proponemos la siguiente solucion:

1. Antes de aplicar modelos de regresion, entrenaremos un **modelo de clasificacion** cuyo objetivo sera determinar si el dato es o no, un outlier. El criterio para definir si un dato es o no un outlier sera especificado mas adelante. Ahora, si logramos una buena precision en este modelo, tendremos nuestro dataset original dividido en dos sub-datasets: uno con datos "confiables", y otro con datos feos.
2. Una vez que tenemos nuestros dos sub-datasets, aplicaremos el proceso que venimos aplicando al dataset original hasta ahora a cada uno de estos, generando features para cada uno de manera independiente y tratando el problema como dos sub problemas. La idea es que el modelo que trata con los datos confiables no se "maree" con los outliers, e intentar predecir estos con un modelo que este entrenado para tal fin.
3. Finalmente, predecimos los precios de las propiedades en test siguiendo este procedimiento y unimos todas las predicciones para dar una prediccion final.
<hr>

### IMPORTANTE:
**NO** es necesario **correr todo el notebook para poder entrenar**. Se pueden **ejecutar por pasos**, ya que cada paso termina guardando los resultados obtenidos.
<hr>

# Paso 1: Entrenar modelo de clasificacion

In [5]:
train,test = pre_processing.load_featured_datasets()

In [6]:
train = feature_selection.get_selected_dataframe(train)
test = feature_selection.get_selected_dataframe(test, precio=False)

In [7]:
# Nuestro criterio para detectar un outlier sera que su precio sea mayor al precio promedio + el desvio.
mean = train['precio'].describe()[1]
std = train['precio'].describe()[2]

In [8]:
def bin_std(x, sup):
    if (x<sup):
        return 1
    return 0

In [9]:
train['target'] = train['precio'].map(lambda x: bin_std(x, mean+std))

In [10]:
train['target'].value_counts()

1    206838
0     33162
Name: target, dtype: int64

In [11]:
# Vemos que con este criterio, tenemos 33k de datos "outliers" o "con precios muy altos", que pueden confundir
# al modelo de regresion.

In [12]:
# Entrenamos el modelo de clasificacion:

In [13]:
train_2 = train.drop('precio', axis=1)

In [14]:
X = train_2.drop('target', axis=1)
Y = train_2['target']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [17]:
import lightgbm as lgb

In [33]:
lightgbm = {'objective': 'binary',
            'metric':'auc',
            'num_leaves': 75,
            'max_depth': 7,
            'min_split_gain': 0.01,
            'min_child_weight': 5.00001,
            'learning_rate': 0.05,
            'lambda_l2': 0,
            'feature_fraction': 0.7000000000000001,
            'bagging_fraction': 1.0,
            'n_estimators': 99999,
            'early_stopping_round': 500}

In [34]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(lightgbm, d_train, valid_sets=watchlist, verbose_eval=500)

Training until validation scores don't improve for 500 rounds
[500]	valid_0's auc: 0.978176
[1000]	valid_0's auc: 0.978976
[1500]	valid_0's auc: 0.979177
Early stopping, best iteration is:
[1468]	valid_0's auc: 0.979189


Tenemos nuestro clasificador listo para separar.
Vemos que hemos obtenido un **AUC de 0.979189**, es decir que el modelo sera capaz de detectar outliers con una precision altisima. Esto es importante ya que el precio final predecido dependera muchisimo de esta separacion temprana.

Vamos entonces a **clasificar los datasets originales** en dos datasets y guardarlos.

In [35]:
test_predicted = reg.predict(test)

In [36]:
f = np.vectorize(lambda x: 1 if (x>0.5) else 0)
test_predicted = f(test_predicted)

In [37]:
test['target'] = test_predicted

In [38]:
test['target'].value_counts()

1    52355
0     7645
Name: target, dtype: int64

In [39]:
train['target'].value_counts()

1    206838
0     33162
Name: target, dtype: int64

In [40]:
# Ahora lo que queremos es separar en dos el problema: MODELO para los datos confiables, y otro modelo
# para los datos no confiables, y luego juntar todo.

In [46]:
train_outliers = train[['target']]
test_outliers = test[['target']]

In [57]:
outliers = train_outliers.append(test_outliers)

In [58]:
outliers.to_csv('data/outliers.csv')

# Entrenamos el modelo para los datos confiables

In [4]:
train,test = pre_processing.load_featured_reliable_datasets()

In [5]:
train = feature_selection.get_selected_dataframe_reliable(train)
test = feature_selection.get_selected_dataframe_reliable(test, precio=False)

In [6]:
# Entrenamos el modelo:
X = train.drop('precio', axis=1)
Y = train['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [15]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14,
    'num_leaves': 120,
    #'learning_rate': 0.02,
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 100}
n_estimators=20000

In [16]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, valid_sets=watchlist, verbose_eval=1000)

Training until validation scores don't improve for 100 rounds
[1000]	valid_0's l1: 305724
[2000]	valid_0's l1: 300199
[3000]	valid_0's l1: 297637
[4000]	valid_0's l1: 296245
[5000]	valid_0's l1: 295494
[6000]	valid_0's l1: 295073
Early stopping, best iteration is:
[6351]	valid_0's l1: 294946


In [17]:
# Realizamos las predicciones
pred_reliable = reg.predict(test.values)

In [18]:
test_a = test.copy()

In [19]:
test_a['precio'] = pred_reliable

In [20]:
test_a = test_a.reset_index()[['id', 'precio']]

In [21]:
# tenemos test a
test_a.head()

,id,precio
0,51775,1.033856e+06
1,115253,2.287917e+06
2,299321,1.108731e+06
3,173570,6.314746e+05
4,30862,1.305523e+06


# Entrenamos el modelo para los datos no confiables

In [24]:
train,test = pre_processing.load_featured_dangerous_datasets()

In [25]:
train = feature_selection.get_selected_dataframe_dangerous(train)
test = feature_selection.get_selected_dataframe_dangerous(test, precio=False)

In [48]:
# Entrenamos el modelo:
X = train.drop('precio', axis=1)
Y = train['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.05, random_state=seed)

In [49]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 11,
    'num_leaves': 70,
    'learning_rate': 0.02,
    #'learning_rate': 0.05,
    'verbose': 0} 
    #'early_stopping_round': 100}
n_estimators=10000

In [50]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, valid_sets=watchlist, verbose_eval=1000)

[50]	valid_0's l1: 1.14059e+06
[100]	valid_0's l1: 1.01839e+06
[150]	valid_0's l1: 966292
[200]	valid_0's l1: 938944
[250]	valid_0's l1: 924212
[300]	valid_0's l1: 915103
[350]	valid_0's l1: 909397
[400]	valid_0's l1: 905823
[450]	valid_0's l1: 903137
[500]	valid_0's l1: 900690
[550]	valid_0's l1: 897392
[600]	valid_0's l1: 895082
[650]	valid_0's l1: 893048
[700]	valid_0's l1: 890852
[750]	valid_0's l1: 888307
[800]	valid_0's l1: 887165
[850]	valid_0's l1: 884856
[900]	valid_0's l1: 882413
[950]	valid_0's l1: 880793
[1000]	valid_0's l1: 878979
[1050]	valid_0's l1: 876765
[1100]	valid_0's l1: 875251
[1150]	valid_0's l1: 874628
[1200]	valid_0's l1: 873690
[1250]	valid_0's l1: 872420
[1300]	valid_0's l1: 871978
[1350]	valid_0's l1: 871587
[1400]	valid_0's l1: 871081
[1450]	valid_0's l1: 870351
[1500]	valid_0's l1: 869504
[1550]	valid_0's l1: 869061
[1600]	valid_0's l1: 868316
[1650]	valid_0's l1: 867653
[1700]	valid_0's l1: 866957
[1750]	valid_0's l1: 866664
[1800]	valid_0's l1: 866094
[1

In [52]:
# Realizamos las predicciones
pred_dangerous = reg.predict(test.values)

In [53]:
test_b = test.copy()

In [54]:
test_b['precio'] = pred_dangerous

In [55]:
test_b = test_b.reset_index()[['id', 'precio']]

In [56]:
# tenemos test b
test_b.head()

,id,precio
0,4941,7.231153e+06
1,262957,6.182953e+06
2,253578,7.178361e+06
3,62134,7.759135e+06
4,277353,5.869489e+06


# Join

In [57]:
test_a['id'].nunique()

52355

In [58]:
test_b['id'].nunique()

7645

In [59]:
test = test_a.append(test_b)

In [60]:
test['id'].nunique()

60000

# Submit

In [63]:
ids = test['id'].values
valores = test['precio'].values

In [64]:
escribir_respuesta(ids, valores)